# Greeks demo (European options on futures, Black-76)

We start from a single **call** option instance and, for each Greek,
1) explain what it means, 2) compute it analytically using `volkit.future`,
3) approximate it numerically using a small **central difference** bump, and
4) print the values and the approximation error.


In [1]:
import numpy as np
from volkit.future import (
    price_euro_future,
    delta_euro_future,
    gamma_euro_future,
    vega_euro_future,
    theta_euro_future,
    rho_euro_future,
    dual_delta_euro_future,
    vanna_euro_future,
    vomma_euro_future,
    lambda_euro_future,
)

np.set_printoptions(precision=10, suppress=True)

# Base parameters (edit me)
F = 100.0
K = 100.0
T = 0.5    # years
r = 0.02   # cc rate
sigma = 0.20
cp = 'call'

## Helper: central difference utilities
- Central difference for first derivative: $f'(x) \approx \dfrac{f(x+h)-f(x-h)}{2h}$
- Central difference for second derivative: $f''(x) \approx \dfrac{f(x+h)-2f(x)+f(x-h)}{h^2}$
- Mixed partial (vanna): $\partial^2 f/\partial x\,\partial y \approx \dfrac{f(x+h,y+k)-f(x+h,y-k)-f(x-h,y+k)+f(x-h,y-k)}{4hk}$


In [2]:
def price(F_, K_, T_, r_, sigma_, cp_='call'):
    return price_euro_future(F_, K_, T_, r_, sigma_, cp_)

def central_first_wrt_F(F, K, T, r, sigma, h):
    return (price(F+h, K, T, r, sigma) - price(F-h, K, T, r, sigma)) / (2.0*h)

def central_second_wrt_F(F, K, T, r, sigma, h):
    return (price(F+h, K, T, r, sigma) - 2.0*price(F, K, T, r, sigma) + price(F-h, K, T, r, sigma)) / (h*h)

def central_first_wrt_sigma(F, K, T, r, sigma, k):
    return (price(F, K, T, r, sigma+k) - price(F, K, T, r, sigma-k)) / (2.0*k)

def central_first_wrt_T(F, K, T, r, sigma, t):
    # ensure positive times
    t = min(t, 0.5*T)
    return (price(F, K, T+t, r, sigma) - price(F, K, T-t, r, sigma)) / (2.0*t)

def central_first_wrt_r(F, K, T, r, sigma, q):
    return (price(F, K, T, r+q, sigma) - price(F, K, T, r-q, sigma)) / (2.0*q)

def central_first_wrt_K(F, K, T, r, sigma, k):
    return (price(F, K+k, T, r, sigma) - price(F, K-k, T, r, sigma)) / (2.0*k)

def mixed_F_sigma(F, K, T, r, sigma, h, k):
    return (
        price(F+h, K, T, r, sigma+k) - price(F+h, K, T, r, sigma-k)
        - price(F-h, K, T, r, sigma+k) + price(F-h, K, T, r, sigma-k)
    ) / (4.0*h*k)

def second_wrt_sigma(F, K, T, r, sigma, k):
    return (price(F, K, T, r, sigma+k) - 2.0*price(F, K, T, r, sigma) + price(F, K, T, r, sigma-k)) / (k*k)

def elastic_numeric(F, K, T, r, sigma, h):
    # lambda = dV/dF * F / V  ~ log-derivative of price wrt F
    V = price(F, K, T, r, sigma)
    V_plus = price(F*(1+h), K, T, r, sigma)
    V_minus = price(F*(1-h), K, T, r, sigma)
    dlogV_dlogF = (np.log(V_plus) - np.log(V_minus)) / (2.0*np.log(1+h))
    return dlogV_dlogF


## Baseline price
This is the present value (discounted) of the option under Black-76.

In [3]:
V0 = price(F, K, T, r, sigma, cp)
print('Price:', V0)

Price: 5.581106724604814


## Delta — sensitivity to futures price (∂V/∂F)
How much the option price moves per 1-unit change in the **futures** price.

In [4]:
analytical = delta_euro_future(F, K, T, r, sigma, cp)
hF = max(1e-4*F, 1e-4)
numerical = central_first_wrt_F(F, K, T, r, sigma, hF)
print('Delta  analytic :', analytical)
print('Delta  numerical:', numerical)
print('abs err:', abs(analytical - numerical))

Delta  analytic : 0.5229304504976081
Delta  numerical: 0.5229304435331983
abs err: 6.964409826615281e-09


## Gamma — curvature w.r.t. futures (∂²V/∂F²)
How delta changes as **F** changes; curvature of price in **F**.

In [5]:
analytical = gamma_euro_future(F, K, T, r, sigma, cp)
hF = max(1e-3*F, 1e-3)
numerical = central_second_wrt_F(F, K, T, r, sigma, hF)
print('Gamma  analytic :', analytical)
print('Gamma  numerical:', numerical)
print('abs err:', abs(analytical - numerical))

Gamma  analytic : 0.027859055399082604
Gamma  numerical: 0.027858948026349133
abs err: 1.0737273347175003e-07


## Vega — sensitivity to volatility (∂V/∂σ)
Price change for a small change in **volatility** (per 1.00 vol, not per 1%).

In [6]:
analytical = vega_euro_future(F, K, T, r, sigma, cp)
k = max(1e-4*sigma, 1e-5)
numerical = central_first_wrt_sigma(F, K, T, r, sigma, k)
print('Vega   analytic :', analytical)
print('Vega   numerical:', numerical)
print('abs err:', abs(analytical - numerical))

Vega   analytic : 27.85905539908261
Vega   numerical: 27.859055398926632
abs err: 1.559783413540572e-10


## Theta — sensitivity to time (∂V/∂T)
Change in price for a 1-year increase in **time to maturity** (holding F constant).

In [13]:
analytical = theta_euro_future(F, K, T, r, sigma, cp)
t = max(1e-4*T, 1e-5)
numerical = -central_first_wrt_T(F, K, T, r, sigma, t)
print('Theta  analytic :', analytical)
print('Theta  numerical:', numerical)
print('abs err:', abs(analytical - numerical))

Theta  analytic : -5.683433214308618
Theta  numerical: -5.4601889525063285
abs err: 0.2232442618022894


## Rho — sensitivity to interest rate (∂V/∂r)
Change in price for a 1.00 (100 percentage points) change in **r** (continuous compounding).

In [8]:
analytical = rho_euro_future(F, K, T, r, sigma, cp)
q = max(1e-4*max(abs(r), 1.0), 1e-5)  # absolute bump on r
numerical = central_first_wrt_r(F, K, T, r, sigma, q)
print('Rho    analytic :', analytical)
print('Rho    numerical:', numerical)
print('abs err:', abs(analytical - numerical))

Rho    analytic : -2.790553362302407
Rho    numerical: -2.7905533634653423
abs err: 1.1629355256559393e-09


## Dual delta — sensitivity to strike (∂V/∂K)
Change in price per 1-unit change in **strike**.

In [9]:
analytical = dual_delta_euro_future(F, K, T, r, sigma, cp)
k = max(1e-4*K, 1e-3)
numerical = central_first_wrt_K(F, K, T, r, sigma, k)
print('DualΔ  analytic :', analytical)
print('DualΔ  numerical:', numerical)
print('abs err:', abs(analytical - numerical))

DualΔ  analytic : -0.46711938325156
DualΔ  numerical: -0.4671193902171744
abs err: 6.965614418596999e-09


## Vanna — cross sensitivity (∂²V/∂F∂σ)
How **vega** changes with **F** (or **delta** with σ).

In [10]:
analytical = vanna_euro_future(F, K, T, r, sigma, cp)
hF = max(1e-3*F, 1e-3)
kS = max(1e-3*sigma, 1e-4)
numerical = mixed_F_sigma(F, K, T, r, sigma, hF, kS)
print('Vanna  analytic :', analytical)
print('Vanna  numerical:', numerical)
print('abs err:', abs(analytical - numerical))

Vanna  analytic : 0.13929527699541305
Vanna  numerical: 0.1392987765913567
abs err: 3.4995959436456836e-06


## Vomma (volga) — curvature in volatility (∂²V/∂σ²)
How **vega** changes with σ (second derivative in volatility).

In [11]:
analytical = vomma_euro_future(F, K, T, r, sigma, cp)
k = max(1e-3*sigma, 1e-4)
numerical = second_wrt_sigma(F, K, T, r, sigma, k)
print('Vomma  analytic :', analytical)
print('Vomma  numerical:', numerical)
print('abs err:', abs(analytical - numerical))

Vomma  analytic : -0.6964763849770655
Vomma  numerical: -0.6964763432648624
abs err: 4.1712203135979564e-08


## Lambda (elasticity) — leverage of price to F
Elasticity $\lambda = \frac{F}{V}\,\Delta$ equals the log-derivative of price wrt **F**.

In [12]:
analytical = lambda_euro_future(F, K, T, r, sigma, cp)
h = 1e-3
numerical = elastic_numeric(F, K, T, r, sigma, h)
print('Lambda analytic :', analytical)
print('Lambda numerical:', numerical)
print('abs err:', abs(analytical - numerical))

Lambda analytic : 9.369655093535874
Lambda numerical: 9.374367012900015
abs err: 0.004711919364140726
